In [19]:
import pybamm

parameter_values = pybamm.ParameterValues.create_from_bpx(r"C:\Users\markv\Documents\Repos\2024\BatterySimulator_v2\BatterySimulator_SimulationBackend\Li-Ion-Battery-Simulator\BatterySimulator\Simulation\LFP\lfp_18650_cell_BPX.json")
parameter_values

{'Ambient temperature [K]': 298.15,
 'Boltzmann constant [J.K-1]': 1.380649e-23,
 'Cation transference number': 0.259,
 'Cell cooling surface area [m2]': 0.00431,
 'Cell volume [m3]': 1.7e-05,
 'Current function [A]': 2.0,
 'Density [kg.m-3]': 1940.0,
 'Electrode area [m2]': 0.08959998,
 'Electrode height [m]': 0.2993325575342582,
 'Electrode width [m]': 0.2993325575342582,
 'Electrolyte conductivity [S.m-1]': <function _bpx_to_param_dict.<locals>._conductivity at 0x0000020555D31300>,
 'Electrolyte conductivity activation energy [J.mol-1]': 17100.0,
 'Electrolyte diffusivity [m2.s-1]': <function _bpx_to_param_dict.<locals>._electrolyte_diffusivity at 0x0000020555D33380>,
 'Electrolyte diffusivity activation energy [J.mol-1]': 17100.0,
 'Electron charge [C]': 1.602176634e-19,
 'External surface area [m2]': 0.00431,
 'Faraday constant [C.mol-1]': 96485.33212,
 'Ideal gas constant [J.K-1.mol-1]': 8.314462618,
 'Initial concentration in electrolyte [mol.m-3]': 1000.0,
 'Initial concentrati

In [21]:
# could implement a way to search what parameter you want to edit.
parameter_values.search("voltage")


Lower voltage cut-off [V]	2.0
Open-circuit voltage at 0% SOC [V]	2.0
Open-circuit voltage at 100% SOC [V]	3.65
Upper voltage cut-off [V]	3.65


In [37]:
# adjusting
parameter_values["Current function [A]"] = 10
parameter_values["Open-circuit voltage at 100% SOC [V]"] = 4.2

In [38]:
# Create a DFN model
model = pybamm.lithium_ion.DFN()

In [39]:
# solver
solver = pybamm.CasadiSolver("safe", atol=1e-6, rtol=1e-6)

# Prevent solver failure if interpolant bounds are exceeded by a negligible amount
solver._on_extrapolation = "warn"

In [40]:
# Define an experiment
experiment = pybamm.Experiment(
    [
        (
            "Discharge at C/5 for 10 hours or until 2.5 V",
            "Rest for 1 hour",
            "Charge at 1 A until 3.5 V",
            "Hold at 3.5 V until 10 mA",
            "Rest for 1 hour",
        ),
    ]
    * 2
)

In [41]:
# Create a simulation object
sim = pybamm.Simulation(model, parameter_values=parameter_values, solver=solver, experiment=experiment)

In [42]:
# Simulate
sim.solve()

At t = 238.258 and h = 5.10856e-13, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 238.258 and h = 1.11737e-18, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 118.258 and h = 7.54588e-18, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 233.714 and h = 2.1527e-07, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 236.506 and h = 1.21426e-09, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 135.305 and h = 1.82393e-13, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 135.305 and h = 1.0935e-13, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 257.867 and h = 3.58477e-09, the corrector convergence failed repeatedly or with |h| = hmin.


In [43]:
# Plot results
sim.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=19.093923303580432, step=0.19093923303580432…

In [49]:
parameter_values = pybamm.ParameterValues.create_from_bpx(r"C:\Users\markv\Documents\Repos\2024\BatterySimulator_v2\BatterySimulator_SimulationBackend\Li-Ion-Battery-Simulator\BatterySimulator\Simulation\LFP\lfp_18650_cell_BPX.json")

NMC=(r"C:\Users\markv\Documents\Repos\2024\BatterySimulator_v2\BatterySimulator_SimulationBackend\Li-Ion-Battery-Simulator\BatterySimulator\Simulation\NMC\AE_gen1_BPX.json")
LFP=(r"C:\Users\markv\Documents\Repos\2024\BatterySimulator_v2\BatterySimulator_SimulationBackend\Li-Ion-Battery-Simulator\BatterySimulator\Simulation\LFP\lfp_18650_cell_BPX.json")

# OOP SUCKS

In [77]:
import pybamm

class SetupSimulation:
    def __init__(self, chemistry, electrochemical_model, solver) -> None:
        self.chemistry = chemistry
        self.adjusted_parameters = {} # Placeholder dictionary for user-adjusted parameters.
        self.electrochemical_model = electrochemical_model # electrochemistry-based lithium-ion battery model 
        self.solver = solver 
        # could put cell_type to help filter dupe chemistries?

    ADJUSTABLE_PARAMETERS = [
            "Ambient temperature [K]",
            "Initial temperature [K]",
            "Lower voltage cut-off [V]",
            "Upper voltage cut-off [V]",
            "Current function [A]",
        ]
    
    # This would be something like localhost:8080/chemistry=LFP/
    def set_battery_chemistry(self, chemistry):
        if chemistry == "LFP":
            # parameter_values = pybamm.ParameterValues.create_from_bpx(LFP)
            parameter_values = pybamm.ParameterValues("Chen2020")
            return parameter_values
        elif chemistry == "NMC":
            # parameter_values = pybamm.ParameterValues.create_from_bpx(NMC)
            parameter_values = pybamm.ParameterValues("Chen2020")
            return parameter_values
        
    # This would be something like localhost:8080/chemistry=LFP/model=dfn
    def set_electrochemical_model(self, electrochemical_model="DFN"): # default will be DFN, idk what it does and I will worry about this when it may matter - maybe
        if electrochemical_model == "DFN":
            return pybamm.lithium_ion.DFN()
        elif electrochemical_model == "SPM":
            return pybamm.lithium_ion.SPM()
        elif electrochemical_model == "SPMe":
            return pybamm.lithium_ion.SPMe()
        else:
            raise ValueError(f"Unknown model type: {electrochemical_model} Please pick DFN, SPM or SPMe")
        
    # # This would be something like localhost:8080/chemistry=LFP/model=dfn/adjustedParameters=true
    # def set_adjusted_parameter_values(self, adjusted_parameters):
    #     parameter_values.update(adjusted_parameters)
    
    # # This would be something like localhost:8080/chemistry=LFP/model=dfn/adjustedParameters=true/solver=casadi
    def set_solver(self, solver):
        solver = pybamm.CasadiSolver("safe", atol=1e-6, rtol=1e-6)
        # Prevent solver failure if interpolant bounds are exceeded by a negligible amount
        solver._on_extrapolation = "warn"
        return solver

# from SetupSimulation import SetupSimulation - not needed in notebook
class SimulateBattery:
    def __init__(self, chemistry, electrochemical_model, solver):
        self.chemistry = chemistry
        self.electrochemical_model = electrochemical_model
        self.solver = solver
        self.setup_simulation = SetupSimulation(chemistry, electrochemical_model, solver)

    def simulate_battery_base_model(self):
        parameter_values = self.setup_simulation.set_battery_chemistry(self.chemistry)
        model = self.setup_simulation.set_electrochemical_model(self.electrochemical_model)
        solver = self.setup_simulation.set_solver(self.solver)

        simulation = pybamm.Simulation(model, parameter_values=parameter_values, solver=solver)

        # Example: Simulating for a specific time range
        time_span = [0, 3700]  # Adjust this according to your needs
        solution = simulation.solve(time_span)

        return solution

if __name__ == "__main__":
    sim = SimulateBattery(chemistry="LFP", electrochemical_model="DFN", solver="CasadiSolver")
    simulation = sim.simulate_battery_base_model()
    
    simulation.plot()

interactive(children=(FloatSlider(value=0.0, description='t', max=3555.448289013619, step=35.55448289013619), …

ModuleNotFoundError: No module named 'SetupSimulation'